## 写在前面， 
1. numpy 用这个版本 numpy-1.19.5， 20.x会报错
2. split_sequences # 构造多元监督学习型数据 这个函数 不要对它的名字所蒙蔽，看不懂可以不看， 在你看完硕士女+博士男的配对后，就明白他的功能了。
3. 数据归一化标准化处理都是障眼法，这段代码有助于你理解lstm的本质。
4. 完全看懂这个，再把归一化，反归一化放进来即可。
-----------------------------------------------
拔高：
5. 当你完成看懂这段代码后，再来尝试回答这个问题：  过去m天 n个特征 决定今天的q个特征 这如何重构数据集， 也就是split_sequences 高度抽象的代码 还不是那么好写，虽然很短的不几行。 
决定今天的q 个特征， 所以喽 # 输出一个数字 所以就写死了 1  model.add(Dense(1)) 这里就是q了 而不是1啦。

In [ ]:
# 导入必备库
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense


In [20]:
# 构造多元监督学习型数据
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# 获取待预测数据的位置
		end_ix = i + n_steps
		# 如果待预测数据超过序列长度，构造完成
		if end_ix > len(sequences)-1:
			break
		# 取前n_steps行数据的前5列作为输入X，第n_step行数据的最后一列作为输出y
		seq_x, seq_y = sequences[i:end_ix, :5], sequences[end_ix, 5:]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [21]:
# 定义序列，构建数据
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([11, 21, 31, 41, 51, 61, 71, 81, 91])
in_seq3 = array([12, 22, 32, 42, 52, 62, 72, 82, 92])
in_seq4 = array([13, 23, 33, 43, 53, 63, 73, 83, 93])
in_seq5 = array([14, 24, 34, 44, 54, 64, 74, 84, 94])
out_seq = array([15, 25, 35, 45, 55, 65, 75, 85, 95])

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
in_seq5 = in_seq5.reshape((len(in_seq5), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, in_seq3,in_seq4,in_seq5,out_seq))

In [22]:
# 到这里就构建好了，dataset和你的csv是一样的，对不对 10, 11, 12, 13, 14->25    20, 21, 22, 23, 24->35 ...
dataset

array([[10, 11, 12, 13, 14, 15],
       [20, 21, 22, 23, 24, 25],
       [30, 31, 32, 33, 34, 35],
       [40, 41, 42, 43, 44, 45],
       [50, 51, 52, 53, 54, 55],
       [60, 61, 62, 63, 64, 65],
       [70, 71, 72, 73, 74, 75],
       [80, 81, 82, 83, 84, 85],
       [90, 91, 92, 93, 94, 95]])

In [23]:
dataset.shape  
# 9行6列的矩阵， 要记住这个形状， 9行6列， 因为下面我会提问。

(9, 6)

In [24]:
# 你的csv表 0-4是特征，5是预测值， 但要注意的是 第一行的0-4，预测的是第二行的5。 第二行的0-4 预测第三行的5
# 也就是说 10, 11, 12, 13, 14--》25  20, 21, 22, 23, 24-》35....
# 所以喽，所谓的# 构造多元监督学习型数据split_sequences 这个函数，就是为了完成这个回归任务的分割，  10, 11, 12, 13, 14--》25  20, 21, 22, 23, 24-》35.... 

In [25]:
# choose a number of time steps
# 设置时间序列步长， 因为定义的是 昨天4特征预测今天1特征 所以就是 1啦， 同理可以是2，3，4.
n_steps = 1
# 输入数据形状X(9,1,5), 9组数据，每组1个时间步，每个时间步5个特征,y(8,1)，8组数据，每组1个特征值：
X, y = split_sequences(dataset, n_steps)
# 定义特征值，直接利用X(6,3,3)中的第3位(特征值)赋值即可
n_features = X.shape[2]

In [26]:
X.shape, y.shape
# 等等，为什么变成 8行了， 这是时间序列预测的特点， 用过去一天的5个特征预测今天的1个特征。
# 也就是说 决定第一行的最后一列的5个特征是缺失的，而最后一行的5个特征啥也决定不了。
# 你也可以这样理解，本科女找硕士男， 硕士女找博士男  所以博士女和本科男就都是光棍。  
# 这时候再回看这个函数， 构造多元监督学习型数据 def split_sequences(sequences, n_steps): 你就明白他的作用了， 因为现在 dataset是 本科男女， 硕士男女，博士男女 占成一排排，他不能做lstm预测
# 经过这个函数， 就成了 本科女找硕士男， 硕士女找博士男  长度也有原来的 3行变成了2行。


((8, 1, 5), (8, 1))

In [68]:
model = Sequential()
# 输入 n_steps = 1 （前一天）,  n_features = 5 (5 特征)
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
# 输出一个数字 所以就写死了 1 
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [69]:
# 到这里 你就搞清楚了，lstm的所谓时间序列的含义， 在股票预测的角度，就是过去决定现在，  过去m天 n个特征 决定今天的q 个特征， 就可以构建回归配对的任务来做了啊（本质就是硕士女，博士男的配对。。。）。
# validation_data=(X, y) 还是用的训练集，偷懒而已
model.fit(X, y, epochs=100, batch_size=2, verbose=2, validation_data=(X, y),shuffle=False)

Epoch 1/100
4/4 - 3s - loss: 4041.9963 - val_loss: 3973.7622
Epoch 2/100
4/4 - 0s - loss: 3883.2280 - val_loss: 3807.4819
Epoch 3/100
4/4 - 0s - loss: 3688.6443 - val_loss: 3583.7666
Epoch 4/100
4/4 - 0s - loss: 3408.9253 - val_loss: 3255.7119
Epoch 5/100
4/4 - 0s - loss: 2997.4995 - val_loss: 2772.3911
Epoch 6/100
4/4 - 0s - loss: 2424.9377 - val_loss: 2125.5610
Epoch 7/100
4/4 - 0s - loss: 1701.9570 - val_loss: 1343.5026
Epoch 8/100
4/4 - 0s - loss: 888.9503 - val_loss: 539.2888
Epoch 9/100
4/4 - 0s - loss: 261.7368 - val_loss: 100.0602
Epoch 10/100
4/4 - 0s - loss: 160.7822 - val_loss: 162.1177
Epoch 11/100
4/4 - 0s - loss: 234.6162 - val_loss: 214.0104
Epoch 12/100
4/4 - 0s - loss: 174.3830 - val_loss: 137.9758
Epoch 13/100
4/4 - 0s - loss: 85.1178 - val_loss: 70.2100
Epoch 14/100
4/4 - 0s - loss: 48.2061 - val_loss: 52.0884
Epoch 15/100
4/4 - 0s - loss: 51.6742 - val_loss: 53.1922
Epoch 16/100
4/4 - 0s - loss: 52.0249 - val_loss: 38.4634
Epoch 17/100
4/4 - 0s - loss: 52.6422 - val

In [70]:
# 构建一个序列输入特征， 他们的y应该回归成115 对不对。
x_input = array([100, 101, 102, 103, 104])  

x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[116.49751]]


In [72]:
yhat.item()

116.49751281738281

In [ ]:
# 116 也可以了啊 和115差不多啊